In [1]:
'''%pip install setuptools==70.3.0  # apparently setuptools < 71.0.0 is required for a required package of openml to install on here
%pip install openml
%pip install numpy
%pip install pandas
# cf. https://pytorch.org/get-started/locally/
#%pip install torch torchvision torchaudio  # cuda 12.4, cf. nvidia-smi shell command
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu  # cpu version
%pip install -U scikit-learn
%pip install scipy
%pip install -U pymoo
%pip list'''

'%pip install setuptools==70.3.0  # apparently setuptools < 71.0.0 is required for a required package of openml to install on here\n%pip install openml\n%pip install numpy\n%pip install pandas\n# cf. https://pytorch.org/get-started/locally/\n#%pip install torch torchvision torchaudio  # cuda 12.4, cf. nvidia-smi shell command\n%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu  # cpu version\n%pip install -U scikit-learn\n%pip install scipy\n%pip install -U pymoo\n%pip list'

In [ ]:
from openml import tasks

from classes import EAGGA

import os
import logging

In [10]:
oml_task_ids = [37, 43, 3903, 3904, 3913, 3918, 10093, 9946, 146819, 359955, 189922, 359962, 190392, 167120, 190137, 190410, 168350, 359975, 359972, 146820]
oml_tasks = tasks.get_tasks(oml_task_ids)

oml_datasets = [oml_task.get_dataset() for oml_task in oml_tasks]

# define positive classes
positive_classes = ['tested_positive', '1', True, True, 'yes', True, '2', '2', '1', '2', '1', True, '1', '1', '2', '1', '2', 'Anomaly', '1', '2']

zipped =list(zip(oml_task_ids, oml_datasets, positive_classes))
zipped = sorted(zipped, key=lambda item: (item[1].get_data()[0].shape[1], item[1].get_data()[0].shape[0]))  # order ascending by # of features, tiebreaker is # of samples
print([(id, ds.name, class_pos, ds.get_data()[0].shape) for id, ds, class_pos in zipped])

c:\workplace\uni\ws-24-25\seminar\ws2425-tab-ml\code\.venv\Lib\site-packages\openml\tasks\functions.py:372: UserWarning: `download_data` will default to False starting in 0.16. Please set `download_data` explicitly to suppress this warning.
  warnings.warn(
c:\workplace\uni\ws-24-25\seminar\ws2425-tab-ml\code\.venv\Lib\site-packages\openml\tasks\functions.py:380: UserWarning: `download_qualities` will default to False starting in 0.16. Please set `download_qualities` explicitly to suppress this warning.
  warnings.warn(


[(359955, 'blood-transfusion-service-center', '2', (748, 5)), (10093, 'banknote-authentication', '2', (1372, 5)), (146820, 'wilt', '2', (4839, 6)), (168350, 'phoneme', '2', (5404, 6)), (37, 'diabetes', 'tested_positive', (768, 9)), (146819, 'climate-model-simulation-crashes', '1', (540, 19)), (359972, 'sylvine', '1', (5124, 21)), (3913, 'kc2', 'yes', (522, 22)), (3918, 'pc1', True, (1109, 22)), (359962, 'kc1', True, (2109, 22)), (3904, 'jm1', True, (10885, 22)), (167120, 'numerai28.6', '1', (96320, 22)), (9946, 'wdbc', '2', (569, 31)), (359975, 'Satellite', 'Anomaly', (5100, 37)), (3903, 'pc3', True, (1563, 38)), (43, 'spambase', '1', (4601, 58)), (190137, 'ozone-level-8hr', '2', (2534, 73)), (190392, 'madeline', '1', (3140, 260)), (190410, 'philippine', '1', (5832, 309)), (189922, 'gina', '1', (3153, 971))]


In [ ]:
hps = {
    'total_layers': (3, 10),
    'nodes_per_hidden_layer': (3, 20),
    'mu': 100,
    'lambda': 10,
    'holdout_train_size': 2/3,
    'cv_k': 5
}

batch_size = 64
min_epochs = 200
patience = 100

secs_per_fold = 2 * 60
secs_total = 8 * 60 * 60

pareto_fronts = list()
for (_, oml_dataset, class_positive) in zipped:
    name = oml_dataset.name
    msg = f'Dataset {name}'
    print(msg)
    logging.info(msg)

    if name in {'diabetes', 'pc3', 'spambase', 'jm1'}:
        msg = f'Skip {name}, already evaluated'
        print(msg)
        logging.info(msg)
        continue

    file_path = os.path.join('export', name)
    
    eagga = EAGGA(
        oml_dataset=oml_dataset,
        class_positive=class_positive,
        hps=hps,
        batch_size=batch_size,
        min_epochs=min_epochs,
        patience=patience,
        secs_per_fold=secs_per_fold,
        secs_total=secs_total,
        file_path=file_path
    )
    #eagga.load_population(0)
    
    pareto_front = eagga.run_eagga()
    pareto_fronts.append(pareto_front)
pareto_fronts

Dataset spambase
Starting init population
